# M1M3 actuator movies
Craig Lage - 07-Jan-24 \
This code plots the M1M3 force actuator errors during a slew.

In [ ]:
import sys, time, os, asyncio
import shlex, subprocess
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker

## Set up the necessary subroutines

In [ ]:
def heatMapZ(df, axp, axs, FATable, index, zmin, zmax):
    types = [['SAA','NA', 'o', 'Z'], ['DAA','Y_PLUS', '^', 'Y_PLUS'], ['DAA','Y_MINUS', 'v', 'Y_MINUS'], \
             ['DAA','X_PLUS', '>', 'X_PLUS'], ['DAA','X_MINUS', '<', 'X_MINUS']]
    axp.set_title("Primary")
    axp.set_xlabel("X position (m)")
    axp.set_ylabel("Y position (m)")

    for [type, orient, marker, label] in types:
        xs = []
        ys = []
        zs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                name=f"primaryCylinderFollowingError{i}"
                zs.append(df.iloc[index][name])
        im = axp.scatter(xs, ys, marker='o', c=zs, cmap='RdBu_r', vmin=zmin, vmax=zmax, s=50, label=label)
    plt.colorbar(im, ax=axp,fraction=0.055, pad=0.02, cmap='RdBu_r')  
    axs.set_title("Secondary")
    axs.set_xlabel("X position (m)")
    axp.set_xlim(-5,5)
    axp.set_ylim(-5,5)
    axs.set_xlim(-5,5)
    axs.set_ylim(-5,5)

    for [type, orient, marker, label] in types:
        if type == 'SAA':
            continue
        xs = []
        ys = []
        zs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                name=f"primaryCylinderFollowingError{i}"
                zs.append(df.iloc[index][name])# - df_zero.iloc[0][name])
        im = axs.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', vmin=zmin, vmax=zmax, s=50, label=label)
    plt.colorbar(im, ax=axs,fraction=0.055, pad=0.02, cmap='RdBu_r')  

def hardPointPlot(df, ax, t, t0, tmin, tmax):
    ax.set_title("Hardpoint forces")
    ax.set_ylabel("measuredForce(N)")
    ax.set_ylim(-3500, 3500)
    times = df['timestamp'].values - t0
    for i in range(6):
        data = df[f'measuredForce{i}'].values
        ax.plot(times, data)
    ax.set_xlim(tmin, tmax)
    ax.set_xticks([])    
    ax.plot([t, t], [-3000, 3000], ls='--', color='black')
    ax.plot([times[0], times[-1]], [3000, 3000], color='red')
    ax.plot([tmin, tmax], [-3000, -3000], color='red')
    ax.plot([tmin, tmax], [1000, 1000], ls='--', color='blue')
    ax.plot([tmin, tmax], [-1000, -1000], ls='--', color='blue')

def TMAPlot(az, el, ax, t, t0, tmin, tmax):
    ax.set_ylabel("TMA Velocity\n(deg/sec)")
    ax.set_ylim(-10,10)
    ax.set_xlabel("Time (sec)")
    times = az['timestamp'] - t0
    azV = az['actualVelocity'].values
    elV = el['actualVelocity'].values
    ax.plot(times, azV, color='blue', label='Az')
    ax.plot(times, elV, color='green', label='El')
    ax.set_xlim(tmin, tmax)
    ax.legend()
    ax.plot([t, t], [-3000, 3000], ls='--', color='black')


## Now generate the frames
### This will take some time

In [ ]:
client = EfdClient('usdf_efd')
dayObs = 20240102
seqNum = 1294
eventMaker = TMAEventMaker()
event = eventMaker.getEvent(dayObs, seqNum)
start = event.begin
end = event.end

dirName = f"/home/c/cslage/u/MTM1M3/movies/actuator_{dayObs}_{seqNum}"
%mkdir -p {dirName}
padStart = 1.0
plotStart = start - TimeDelta(padStart, format='sec') 
padEnd = 0.0
plotEnd = end + TimeDelta(padEnd, format='sec') 
forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", "*", \
                                         plotStart, plotEnd)
hardpoints = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", plotStart, plotEnd)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['*'],  plotStart, plotEnd)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['*'],  plotStart, plotEnd) 

t0 = start.unix_tai
# The value below compensates for the different delays in the
# different databases
t0_az_el = 2.0 * start.unix_tai - az['timestamp'][0] - padStart
tmax = forces['timestamp'][-1] - t0
tmin = -padStart

In [ ]:
# Build the individual frames
zmin = -200.0
zmax = 200.0

fig = plt.figure(figsize=(8,8))
for n in range(len(forces)):
    t = Time(forces.index[n], scale='utc').unix_tai - t0
    t_msec = int(t * 1000)
    fig.suptitle(f"Actuator following errors. T = {t_msec} msec\n {dayObs} - seqNum {seqNum}", y=0.90)
    axp = fig.add_axes((0.1, 0.45, 0.35, 0.35))
    axs = fig.add_axes((0.55, 0.45, 0.35, 0.35))
    heatMapZ(forces, axp, axs, FATable, n, zmin, zmax)
    axHP = fig.add_axes((0.1, 0.23, 0.8, 0.15))
    hardPointPlot(hardpoints, axHP, t, t0, tmin, tmax)
    axTMA = fig.add_axes((0.1, 0.08, 0.8, 0.15))
    TMAPlot(az, el, axTMA, t, t0_az_el, tmin, tmax)
    plt.savefig(f"{dirName}/Frame_{n:05d}.png")
    plt.clf()
plt.close()

## Now build the movie

In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/m1m3_movie_{dayObs}_{seqNum}.mp4\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/m1m3_movie_{dayObs}_{seqNum}.mp4"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()

In [ ]:
client = EfdClient('usdf_efd')
dayObs = 20240108
seqNum = 176
eventMaker = TMAEventMaker()
event = eventMaker.getEvent(dayObs, seqNum)
start = event.begin# - TimeDelta(10.0, format='sec')
end = event.end + TimeDelta(5.0, format='sec')


In [ ]:
index = 100
names = ['AccForce', 'BalForce', 'ElForce', 'AppliedForce', 'MeasuredForce', 'Error']
forceList = []
items = [f"zForces{index}", f"zForces{index}", \
         f"zForces{index}", f"zForces{index}", f"primaryCylinderForce{index}", \
         f"primaryCylinderFollowingError{index}"]
accForces = await client.select_time_series("lsst.sal.MTM1M3.appliedAccelerationForces", \
                                            [f"zForces{index}", 'timestamp'], \
                                         start, end)
forceList.append(accForces)
balForces = await client.select_time_series("lsst.sal.MTM1M3.appliedBalanceForces", \
                                    [f"zForces{index}", 'timestamp'], \
                                         start, end)
forceList.append(balForces)
elForces = await client.select_time_series("lsst.sal.MTM1M3.appliedElevationForces", \
                                           [f"zForces{index}", 'timestamp'], \
                                         start, end)
forceList.append(elForces)
appForces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", \
                                            [f"zForces{index}", 'timestamp'], \
                                         start, end)
forceList.append(appForces)
forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", f"primaryCylinderForce{index}", \
                                         start, end)
forceList.append(forces)
errors = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", f"primaryCylinderFollowingError{index}", \
                                         start, end)
forceList.append(errors)

staticStart = start - TimeDelta(0.50) # This is 0.25 days
static_forces = appForces = await client.select_time_series("lsst.sal.MTM1M3.logevent_appliedStaticForces", \
                                            [f"zForces{index}", 'timestamp'], \
                                         staticStart, end)
static_force = static_forces[items[3]].values[-1]
print(static_force)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['timestamp', 'actualPosition'],  start, end) 


In [ ]:
len(static_forces)

In [ ]:
fig, axs = plt.subplots(2,3,figsize=(12,5))
plt.subplots_adjust(hspace=0.8, wspace=0.3)
plt.suptitle(f"Actuator forces {dayObs} - {seqNum} - zForces{index} Azimuth only slew")
counter = 0
for y in range(2):
    for x in range(3):
        ax = axs[y,x]
        ax.set_title(names[counter])
        forceList[counter][items[counter]].plot(ax=ax)
        if counter == 2:
            ax2 = ax.twinx()
            elArc = (el['actualPosition'] - el['actualPosition'].values.min()) * 3600.0
            ax2.set_ylabel("Elevation change (arcsec)", color='red')
            ax.set_ylabel("Force(N)", color='blue')
            elArc.plot(ax = ax2, color='red')
            print(el.values.min(), el.values.max())
            #ax2.set_ylim(
        #ax.set_ylim(0, 1500)
        counter += 1
    
plt.savefig(f"/home/c/cslage/u/MTM1M3/actuator_forces/Actuator_Forces_{dayObs}_{seqNum}_{index}.png")

In [ ]:
plt.suptitle(f"Applied-Acceleration-Balance-Elevation-Static {dayObs} - {seqNum} - zForces{index}")
accLength = len(forceList[0][items[0]].values)
times = forceList[3]['timestamp'].values[0:accLength]
times -= times[0]
diff = forceList[3][items[3]].values[0:accLength] - (forceList[0][items[0]].values[0:accLength] \
                                        + forceList[1][items[1]].values[0:accLength] + forceList[2][items[2]].values[0:accLength])
diff -= static_force
plt.plot(times, diff)
plt.ylabel("Force(N)")
plt.xlabel("Time(sec)")
plt.savefig(f"/home/c/cslage/u/MTM1M3/actuator_forces/Actuator_Force_Differences_{dayObs}_{seqNum}_{index}.png")